In [2]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 0. Read the data

In [3]:
positive = open("../positive.txt", "r").read()
negative = open("../negative.txt", "r").read()

# 1. Text-Preprocessing

Tokenizing

In [4]:
from nltk.tokenize import word_tokenize, sent_tokenize

list_words = word_tokenize(positive) + word_tokenize(negative)

Words cleaning (stopwords, stemming, POS tagging)

In [5]:
from nltk.corpus import stopwords
from string import punctuation

eng_stopwords = stopwords.words('english')

list_words = [word for word in list_words if word not in eng_stopwords]
list_words = [word for word in list_words if word not in punctuation]
list_words = [word for word in list_words if word.isalpha()]

In [6]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')

list_words = [stemmer.stem(word) for word in list_words]

pos tagging for NER

In [7]:
from nltk.tag import pos_tag
tagged = pos_tag(list_words)

['this', 'watch', 'realli', 'good', 'durabl', 'this', 'tv', 'realli', 'aw', 'keep', 'turn']
